In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ( 
     accuracy_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay, PrecisionRecallDisplay
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
import xgboost as xgb
import joblib

In [6]:
# ----- Load CSV with robust path handling

CANDIDATE_PATHS = [
    "../data/hypertension_cleaned.csv",
    "./data/hypertension_cleaned.csv",
    "./hypertension_cleaned.csv",
    "/mnt/data/hypertension_cleaned.csv"
]

for p in CANDIDATE_PATHS:
    if os.path.exists(p):
        data_path = p
        break
else:
    raise FileNotFoundError(f"Could not find hypertension_cleaned.csv in any of: {CANDIDATE_PATHS}")

data = pd.read_csv(data_path)
print(f"Loaded data from: {data_path} -> shape={data.shape}")

Loaded data from: ./data/hypertension_cleaned.csv -> shape=(8835, 7)


In [7]:
data.head()

,Age,BMI,Systolic_BP,Diastolic_BP,Heart_Rate,Gender,Hypertension
0,83,16.6,156,63,91,Male,High
1,37,38.7,102,85,51,Male,High
2,73,26.4,92,102,50,Male,High
3,65,17.3,100,84,75,Female,High
4,85,25.7,177,64,93,Female,High


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8835 entries, 0 to 8834
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8835 non-null   int64  
 1   BMI           8835 non-null   float64
 2   Systolic_BP   8835 non-null   int64  
 3   Diastolic_BP  8835 non-null   int64  
 4   Heart_Rate    8835 non-null   int64  
 5   Gender        8835 non-null   object 
 6   Hypertension  8835 non-null   object 
dtypes: float64(1), int64(4), object(2)
memory usage: 483.3+ KB


### Basic Data Cleaning & Quick Sanity checks

In [9]:
# creating a copy of the data
df = data.copy()